In [2]:
import pyspark
from delta import *


In [3]:
builder=(pyspark.sql.SparkSession.builder.appName("TimeTravel")
         .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension")
         .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
         )

In [4]:
spark=configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/vk/Library/Python/3.9/lib/python/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/vk/.ivy2/cache
The jars for the packages stored in: /Users/vk/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7ff04482-3140-42bb-90de-a7e9d4a74e90;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 91ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   

23/04/16 20:08:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Create Delta Table

In [5]:
data = spark.range(0,5)
data.rdd.getNumPartitions()

8

In [12]:
data.coalesce(1).write.format("delta").save("/tmp/time-travel-delta")


In [13]:
%ls '/tmp/time-travel-delta/'

_delta_log/
part-00000-bd160bfb-f567-46d7-8da0-c7ea4d25cbb8-c000.snappy.parquet


In [14]:
datav1=spark.range(6,10)

In [16]:
datav1.coalesce(1).write.mode("append").format("delta").save("/tmp/time-travel-delta/")

In [17]:
%ls "/tmp/time-travel-delta/"

_delta_log/
part-00000-358d180d-d0eb-49e0-b473-3368f76fbe43-c000.snappy.parquet
part-00000-bd160bfb-f567-46d7-8da0-c7ea4d25cbb8-c000.snappy.parquet


In [18]:
datav2=spark.createDataFrame([(11,),(12,),(13,)]).toDF("id")

In [19]:
datav2.coalesce(1).write.mode("overwrite").format("delta").save("/tmp/time-travel-delta/")

# Read Different Versions of Data

In [21]:
spark.read.format("delta").load("/tmp/time-travel-delta/").show()

+---+
| id|
+---+
| 11|
| 12|
| 13|
+---+



In [23]:
%ls -lrt '/tmp/time-travel-delta/_delta_log/'

total 24
-rw-r--r--@ 1 vk  wheel   985 Apr 16 20:14 00000000000000000000.json
-rw-r--r--@ 1 vk  wheel   642 Apr 16 20:17 00000000000000000001.json
-rw-r--r--@ 1 vk  wheel  1052 Apr 16 20:22 00000000000000000002.json


In [24]:
spark.read.format("delta").option("versionAsof","0").load("/tmp/time-travel-delta/").show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [26]:
spark.read.format("delta").option("versionAsof","1").load("/tmp/time-travel-delta/").show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  6|
|  7|
|  8|
|  9|
+---+



In [ ]:
from delta.tables import DeltaTable

In [28]:
delta_table = DeltaTable.forPath(spark,"/tmp/time-travel-delta/")

In [32]:
delta_table.history().select("version","timestamp","operation","operationParameters").show(truncate=False)

+-------+-----------------------+---------+------------------------------------------+
|version|timestamp              |operation|operationParameters                       |
+-------+-----------------------+---------+------------------------------------------+
|2      |2023-04-16 20:22:28.682|WRITE    |{mode -> Overwrite, partitionBy -> []}    |
|1      |2023-04-16 20:17:33.234|WRITE    |{mode -> Append, partitionBy -> []}       |
|0      |2023-04-16 20:14:41.475|WRITE    |{mode -> ErrorIfExists, partitionBy -> []}|
+-------+-----------------------+---------+------------------------------------------+

